This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XXXXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import collections
from datetime import datetime

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r1 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?column_index=4&start_date=2019-01-08&end_date=2019-01-08&api_key=GvyjkcCS7KPMeNmsvaZc')

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r1.json()

{u'dataset': {u'collapse': None,
  u'column_index': 4,
  u'column_names': [u'Date', u'Close'],
  u'data': [[u'2019-01-08', 74.2]],
  u'database_code': u'FSE',
  u'database_id': 6129,
  u'dataset_code': u'AFX_X',
  u'description': u'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  u'end_date': u'2019-01-08',
  u'frequency': u'daily',
  u'id': 10095370,
  u'limit': None,
  u'name': u'Carl Zeiss Meditec (AFX_X)',
  u'newest_available_date': u'2019-01-11',
  u'oldest_available_date': u'2000-06-07',
  u'order': None,
  u'premium': False,
  u'refreshed_at': u'2019-01-11T22:32:45.336Z',
  u'start_date': u'2019-01-08',
  u'transform': None,
  u'type': u'Time Series'}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
r2 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&order=asc&api_key=GvyjkcCS7KPMeNmsvaZc')
r2.json()['dataset']['column_names']

[u'Date',
 u'Open',
 u'High',
 u'Low',
 u'Close',
 u'Change',
 u'Traded Volume',
 u'Turnover',
 u'Last Price of the Day',
 u'Daily Traded Units',
 u'Daily Turnover']

### 2. Convert the returned JSON object into a Python dictionary.
Creating a dictonary where every date is a key with all the different prices as values in a list. The order of these prices would be same as above printed columns.

In [7]:
AFX_X_dict = collections.OrderedDict()
for x in r2.json()['dataset']['data']:
    AFX_X_dict[datetime.strptime(x[0] , '%Y-%m-%d')]= x[1:]
type(AFX_X_dict)

collections.OrderedDict

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.
From the column list above the first value in the list of every key should be the opening price.

Note: Some of the values in opening prices are not available or are nulls. Since opening values can't be sustiuted by statistical or any previous or next day values, we will be filtering those values out.

In [8]:
opening_prices = []
for key, val in AFX_X_dict.iteritems():
#Filtering out nulls or None  
    if val[0]!=None:
            opening_prices.append(val[0])
print('Highest opening price: {}'.format(max(opening_prices)))
print('Lowest opening price: {}'.format(min(opening_prices)))

Highest opening price: 53.11
Lowest opening price: 34.0


### 4. What was the largest change in any one day (based on High and Low price)?
Since all the change columns are empty or null, we will have to calculate them seprately. The highest and lowest values are at second and thirst position respectively.

Note: The highs and lows don't seem to have any nulls, hence no action is required.

In [9]:
daily_change = []
for key, val in AFX_X_dict.iteritems():
#     if  None in (val[1], val[2]):
#         print(key)
    daily_change.append(val[1]-val[2])
print("Largest change in a day is of {}".format(max(daily_change)))

Largest change in a day is of 2.81


### 5. What was the largest change between any two days (based on Closing Price)?
Closing price is the 4th element of every day. Finding abolute intra day changes.<br>
Making a dicionary iterable helps in getting first value.

In [10]:
intra_day_changes=[]
iter_dict = iter(AFX_X_dict)
last_key = next(iter_dict)
for key in iter_dict:
    intra_day_changes.append(abs(AFX_X_dict[key][3]-AFX_X_dict[last_key][3]))
    last_key = key
print('The largest price change between two days is {}'.format(max(intra_day_changes)))

The largest price change between two days is 2.56


### 6. What was the average daily trading volume during this year?
Trading volume is at the 6th element in the list.

In [11]:
daily_tading_volume = []
for key, val in AFX_X_dict.iteritems():
    daily_tading_volume.append(val[5])
print('Average daily trading volume is {}'.format(float(sum(daily_tading_volume))/len(daily_tading_volume)))

Average daily trading volume is 89124.3372549


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
Using the daily_tading_volume list from above calculating median value.

In [12]:
def median(arr):
    arr= sorted(arr)
    l = len(arr)
    half_length = l//2
    if l<1:
        return None
    elif l%2==0:
        return arr[half_length]
    else:
        return (arr[half_length-1]+arr[half_length+1])/2

median(daily_tading_volume)    

151575.0